## **Ingesta del archivo `results.json`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Crear tabla **f1_raw.results**

In [ ]:
%sql
DROP TABLE IF EXISTS f1_raw.results;
CREATE EXTERNAL TABLE IF NOT EXISTS f1_raw.results(
resultId INT,
raceId INT,
driverId INT,
constructorId INT,
number INT,grid INT,
position INT,
positionText STRING,
positionOrder INT,
points INT,
laps INT,
time STRING,
milliseconds INT,
fastestLap INT,
rank INT,
fastestLapTime STRING,
fastestLapSpeed FLOAT,
statusId STRING)
USING JSON
OPTIONS(path "/mnt/formula1dl/raw/results.json")

### Paso 2 - Leer el archivo JSON

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [ ]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
results_df = spark.read \
.schema(results_schema) \
.json(f"{raw_folder_path}/results.json")
#.json(f"{raw_folder_path}/{v_file_date}/results.json")

### Paso 3 - Renombrar columnas y añadir columnas nuevas

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

### Paso 4 - Eliminar la columna no deseada

In [ ]:
from pyspark.sql.functions import col

In [ ]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

### Paso 5 - Escribir datos en el datalake como parquet y crear la tabla **results** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos el archivo con formato PARQUET en la base de datos "f1_processed" y en la tabla "results"
results_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.results")

In [ ]:
dbutils.notebook.exit("Success")